In [1]:
from haversine import haversine, Unit
from tqdm import tqdm
import numpy as np

from datetime import datetime
from itertools import groupby

min_path_points = 20
max_path_points = 500

In [2]:
def resample_trajectory(x, length=200):
    """
    Resample a trajectory to a fixed length using linear interpolation
    :param x: trajectory to resample
    :param length: length of the resampled trajectory
    :return: resampled trajectory
    """
    len_x = len(x)
    time_steps = np.arange(length) * (len_x - 1) / (length - 1)
    x = x.T
    resampled_trajectory = np.zeros((2, length))
    for i in range(2):
        resampled_trajectory[i] = np.interp(time_steps, np.arange(len_x), x[i])
    return resampled_trajectory.T

In [3]:
import json

with open("../data/rome_edges.json") as f:
    edges_data = json.load(f)

# make u-v as edge key
edges_data = {(d["u"], d["v"]): d for d in edges_data}

In [4]:
import pandas as pd

trajectory_df = pd.read_csv("../data/rome_matched/rome_matched.csv")
trajectory_ids = trajectory_df["id"].tolist()
cpaths = trajectory_df["c-uv"].tolist()
timestamps = trajectory_df["timestamps"].tolist()

In [5]:
from pyproj import Transformer

transformer = Transformer.from_crs("epsg:32633", "epsg:4326")

In [ ]:
# lat_min, lat_max, lon_min, lon_max = np.inf, -np.inf, np.inf, -np.inf

# for cpath in tqdm(cpaths, total=len(trajectory_ids)):
#     cpath = eval(cpath)

#     if isinstance(cpath[0], int):
#         cpath = [cpath]
    
#     for u, v in cpath:
#         edge_row = edges_data.get((u, v), None)
#         assert edge_row is not None, f"Edge ({u}, {v}) not found in edges data."

#         geometry = edge_row["geometry"]
#         geometry = [list(transformer.transform(x, y))[::-1] for x, y in geometry]

#         lats = [coord[1] for coord in geometry]
#         lons = [coord[0] for coord in geometry]

#         lat_min = min(lat_min, min(lats))
#         lat_max = max(lat_max, max(lats))
#         lon_min = min(lon_min, min(lons))
#         lon_max = max(lon_max, max(lons))

In [6]:
lat_min, lat_max, lon_min, lon_max = (41.656612100000004, 42.0926008, 12.258011, 12.7889121) # Rome

def calculate_grid_id(lat, lon, grid_divisions=12):
    lat_idx = int((lat - lat_min) / (lat_max - lat_min) * (grid_divisions - 1))
    lon_idx = int((lon - lon_min) / (lon_max - lon_min) * (grid_divisions - 1))
    
    grid_id = lat_idx * grid_divisions + lon_idx
    return grid_id

In [7]:
def calculate_trajectory_attributes(trajectory, timestamps):
    trip_length = trajectory.shape[0]
    trip_distance = 0.0
    for i in range(trip_length - 1):
        start_lon, start_lat = trajectory[i]
        end_lon, end_lat = trajectory[i + 1]
        trip_distance += haversine((start_lat, start_lon), (end_lat, end_lon), unit=Unit.METERS)

    avg_dis = trip_distance / (trip_length - 1) if trip_length > 1 else 0

    trip_time = timestamps[-1] - timestamps[0]
    avg_speed = trip_distance / trip_time if trip_time > 0 else 0

    dt_object = datetime.fromtimestamp(timestamps[0])
    hour = dt_object.hour
    minute = dt_object.minute
    departure_time = (hour * 60 + minute) // 5

    start_id = calculate_grid_id(trajectory[0][1], trajectory[0][0])
    end_id = calculate_grid_id(trajectory[-1][1], trajectory[-1][0])

    return {
        "departure_time": departure_time,
        "trip_distance": trip_distance,
        "trip_time": trip_time,
        "trip_length": trip_length,
        "avg_dis": avg_dis,
        "avg_speed": avg_speed,
        "start_id": start_id,
        "end_id": end_id
    }

In [8]:
all_attrs, all_resampled_trajectories = [], []

for trajectory_id, cpath, _timestamps in tqdm(zip(trajectory_ids, cpaths, timestamps), total=len(trajectory_ids)):
    cpath = eval(cpath)
    _timestamps = eval(_timestamps)

    if isinstance(cpath[0], int):
        cpath = [cpath]

    path_osmid, path_geometry = [], []
    for u, v in cpath:
        edge_row = edges_data.get((u, v), None)
        assert edge_row is not None, f"Edge ({u}, {v}) not found in edges data."

        edge_osmid = edge_row["osmid"]
        path_osmid += edge_osmid if isinstance(edge_osmid, list) else [edge_osmid]

        geometry = edge_row["geometry"]
        geometry = [list(transformer.transform(x, y))[::-1] for x, y in geometry]
        path_geometry += geometry

    path_osmid = [key for key, _ in groupby(path_osmid)]

    if len(path_osmid) < min_path_points or len(path_osmid) > max_path_points:
        continue

    trajectory = np.array(path_geometry)  # [L, 2]
    resampled_trajectory = resample_trajectory(trajectory, length=200)  # [200, 2]

    attrs = calculate_trajectory_attributes(trajectory, _timestamps)
    
    all_attrs.append(attrs)
    all_resampled_trajectories.append(resampled_trajectory)

100%|██████████| 175063/175063 [06:55<00:00, 421.22it/s]


In [9]:
attributes = [
    "departure_time",
    "trip_distance",
    "trip_time",
    "trip_length",
    "avg_dis",
    "avg_speed",
    "start_id",
    "end_id",
]

stats = {}
for attr in attributes:
    if attr in ["start_id", "end_id", "departure_time"]:
        continue
    values = np.array([item[attr] for item in all_attrs])
    stats[attr] = {"mean": np.mean(values), "std": np.std(values)}

# make as numpy arrays, and normalize numeric attributes
all_attrs_array = []
for item in all_attrs:
    attr_array = []
    for attr in attributes:
        value = item[attr]
        if attr not in ["start_id", "end_id", "departure_time"]:
            value = (value - stats[attr]["mean"]) / stats[attr]["std"]
        attr_array.append(value)
    all_attrs_array.append(attr_array)

all_attrs_array = np.array(all_attrs_array)  # [N, 8]
all_resampled_trajectories_array = np.array(all_resampled_trajectories)  # [N, 200, 2]

In [12]:
from sklearn.model_selection import train_test_split

train_val_inds, test_inds = train_test_split(range(len(all_resampled_trajectories_array)), test_size=0.2, random_state=42)
train_inds, val_inds = train_test_split(train_val_inds, test_size=0.125, random_state=42)

train_traj_array = all_resampled_trajectories_array[train_inds]
val_traj_array = all_resampled_trajectories_array[val_inds]
test_traj_array = all_resampled_trajectories_array[test_inds]

train_traj_attrs = all_attrs_array[train_inds]
val_traj_attrs = all_attrs_array[val_inds]
test_traj_attrs = all_attrs_array[test_inds]

In [13]:
np.save("../data/rome_train_trajectories.npy", train_traj_array)
np.save("../data/rome_val_trajectories.npy", val_traj_array)
np.save("../data/rome_test_trajectories.npy", test_traj_array)

np.save("../data/rome_train_attributes.npy", train_traj_attrs)
np.save("../data/rome_val_attributes.npy", val_traj_attrs)
np.save("../data/rome_test_attributes.npy", test_traj_attrs)

In [16]:
train_traj_array.shape, val_traj_array.shape, test_traj_array.shape

((111863, 200, 2), (15981, 200, 2), (31962, 200, 2))